In [1]:
import socket

In [13]:
# socket listeners are constructed for a particular host and specific port number.
HOST = '127.0.0.1'
PORT = 65432

# the actual socket is initialized using a context manager, which handles closing the
# socket at exit time.
with socket.socket(
    # AF_INET is the IPv4 socket address family. socket.AF_INET6 is the IPv6 socket address family.
    # A socket address family is a communications protocol for the socket. Linux supports a bunch of
    # different communication protocols in addition to IPv4 and IPv6 using sockets.
    socket.AF_INET,
    # The second parameter is the socket type, and it controls the transport layer. SOCK_STREAM
    # is for reliable, well-ordered communication - TCP. SOCK_DGRAM is for unreliable, unordered
    # communication - UDP.
    socket.SOCK_STREAM
) as s:
    # Bind binds the socket listener to the socket.
    s.bind(
        # HOST is the IP address to listen to connections on. If you set this to 127.0.0.1, the loopback
        # address, only packets emitted by this machine will be directed to this socket. If you set this
        # to '0.0.0.0', this port will be bound to *all* IP addresses. If you set this to
        # a specific real IP address, network traffic to that address will go to this socket.
        ('192.168.1.6',
        # PORT is the port number. Numbers below 1024 are reserved, and require superuser permissions to
        # take over. The two most important reserved ports are 80 and 443.
         65432)
    )
    # listen makes this a "listening socket". The only parameter this method has is a configurable value
    # for setting the maximum number of connection requests to enqueue. The precise value is OS-dependent.
    s.listen()
    # accept is a blocking task which does nothing until a request comes in or is poppable off the queue.
    # connect returns a connected socket -- a socket which is actively able to communicate with the client
    # that connected to it. The second component, addr, is the address of the connecting client.
    conn, addr = s.accept()
    with conn:
        print('Connected by', addr)
        # Here we use a busy loop to manage the connection.
        while True:
            # recv is the function for reading out bytes from the data payload
            data = conn.recv(1024)
            # we break the connection (and hence, close the socket) if we recieve no data
            if not data:
                break
            # the data we get we send back
            conn.sendall(data)

Connected by ('192.168.1.6', 65433)


Here is the code for the client, copied from `~/scratch/scripts/client.py`. Run this in a separate process after running the code block above, and see what happens:

In [ ]:
import socket

HOST = '192.168.1.6'
PORT = 65432

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    # If you don't intend to listen with a socket, only to send and recieve data to another socket,
    # it's not necessary to actually bind it, as Python will pick an unoccupied port number for you
    # automatically. So technically this following line of code can be omitted.
    s.bind((HOST, 65433))
    s.connect((HOST, PORT))
    s.sendall(b'Hello, world')
    data = s.recv(1024)

print('Received', repr(data))